# 使用测试

## 话题激发模块、相似度模块及文本存储模块测试

In [ ]:
from history import History
import random
import stimulate

# 示例使用
history = History("example.txt", "This is a test text.")
history.save_text_to_file()

history.file_handler.add_file("example2.txt")  # 添加另一个文件
if stimulate.should_activate():
    # 生成一个全局随机数
    global_random_number = random.randint(1, 100)

    # 根据哈希值和随机数选择一个文件
    selected_file = history.file_handler.select_file(global_random_number)

    if selected_file:
        print(f"选中的文件是: {selected_file}")
    else:
        print("没有可用的文件。")


## 好感度模块测试

In [ ]:
import favorability 
moodvads = [(5, 3, 4), (6, 3, 5), (7, 4, 6), (6, 4, 3)]  # 情绪变化记录
fav = favorability .Favorability(moodvads)
print(f"当前好感度: {fav.get_favorability()}")  # 输出当前好感度

# 添加新情绪
fav.add_change((8, 5, 7))
print(f"更新后好感度: {fav.get_favorability()}")

# 再次添加情绪变化
fav.add_change((4, 3, 2))
print(f"再次更新后好感度: {fav.get_favorability()}")